In [1]:
import nltk
import pandas as pd

In [2]:
fake = pd.read_csv("../input/news-classification/Fake.csv")
genuine = pd.read_csv("../input/news-classification/True.csv")

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
genuine.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['target'] = 0
genuine['target'] = 1

In [7]:
data = pd.concat([fake, genuine], axis=0)
data = data.reset_index(drop=True)
data=data.drop(['subject','date','title'], axis=1)
print(data.columns)
print(data)

Index(['text', 'target'], dtype='object')
                                                    text  target
0      Donald Trump just couldn t wish all Americans ...       0
1      House Intelligence Committee Chairman Devin Nu...       0
2      On Friday, it was revealed that former Milwauk...       0
3      On Christmas day, Donald Trump announced that ...       0
4      Pope Francis used his annual Christmas Day mes...       0
...                                                  ...     ...
44893  BRUSSELS (Reuters) - NATO allies on Tuesday we...       1
44894  LONDON (Reuters) - LexisNexis, a provider of l...       1
44895  MINSK (Reuters) - In the shadow of disused Sov...       1
44896  MOSCOW (Reuters) - Vatican Secretary of State ...       1
44897  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...       1

[44898 rows x 2 columns]


In [8]:
from nltk.tokenize import word_tokenize
data['text']=data['text'].apply(word_tokenize)
data.head()

,text,target
0,"[Donald, Trump, just, couldn, t, wish, all, Am...",0
1,"[House, Intelligence, Committee, Chairman, Dev...",0
2,"[On, Friday, ,, it, was, revealed, that, forme...",0
3,"[On, Christmas, day, ,, Donald, Trump, announc...",0
4,"[Pope, Francis, used, his, annual, Christmas, ...",0


In [9]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer("english", ignore_stopwords=False)
def stem_it(text):
    return [porter.stem(word) for word in text]
data['text']=data['text'].apply(stem_it)
data.head()

,text,target
0,"[donald, trump, just, couldn, t, wish, all, am...",0
1,"[hous, intellig, committe, chairman, devin, nu...",0
2,"[on, friday, ,, it, was, reveal, that, former,...",0
3,"[on, christma, day, ,, donald, trump, announc,...",0
4,"[pope, franci, use, his, annual, christma, day...",0


In [10]:
def stop_it(t):
    dt = [word for word in t if len(word)>2] 
    return dt
data['text']=data['text'].apply(stop_it)
data.head()

,text,target
0,"[donald, trump, just, couldn, wish, all, ameri...",0
1,"[hous, intellig, committe, chairman, devin, nu...",0
2,"[friday, was, reveal, that, former, milwauke, ...",0
3,"[christma, day, donald, trump, announc, that, ...",0
4,"[pope, franci, use, his, annual, christma, day...",0


In [11]:
data['text']=data['text'].apply(' '.join)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.20)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf = TfidfVectorizer( max_df=0.7)

tfidf_train = my_tfidf.fit_transform(X_train)
tfidf_test = my_tfidf.transform(X_test)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model_1 = LogisticRegression(max_iter=900)
model_1.fit(tfidf_train, y_train)
pred_1 = model_1.predict(tfidf_test)
cr1    = accuracy_score(y_test,pred_1)
print(cr1*100)

98.93095768374165


In [16]:
from sklearn.linear_model import PassiveAggressiveClassifier

model = PassiveAggressiveClassifier(max_iter=50)
model.fit(tfidf_train, y_train)
y_pred = model.predict(tfidf_test)
accscore = accuracy_score(y_test, y_pred)
print("accuracy " ,accscore*100)

accuracy  99.61024498886414
